Spatial Analysis:

Conduct spatial analysis techniques such as:

Spatial Clustering: Identify hotspots or clusters of business entities using methods like K-means clustering or DBSCAN.

Spatial Autocorrelation: Determine if there are spatial patterns in the distribution of businesses using Moran's I or Geary's C.



### 🔍 COLUMN OVERVIEW & POTENTIAL USES

| Column Name                 | Description / Use                                                                                   |
| --------------------------- | --------------------------------------------------------------------------------------------------- |
| `Identification number`     | Unique ID for business; useful for joining with other data if needed.                               |
| `Personal number`           | Might be sensitive or personal data – unless necessary, consider dropping or anonymizing.           |
| `Organizational-legal form` | Allows analysis by business structure (LLC, sole proprietor, etc.) – can signal formality or scale. |
| `Name of entity`            | For labeling or filtering; maybe unnecessary for spatial analysis.                                  |
| `Actual address`            | Helpful for verifying location data; could be used for geocoding (if coords missing).               |
| `Name of activity`          | **Key for sectoral analysis** (e.g. retail vs services vs industry).                                |
| `Business size`             | Indicates scale (small/medium/large) – great for economic weighting.                                |
| `address` (duplicate?)      | Might be the same as `Actual address`; clarify or drop if redundant.                                |
| `long`, `lat`               | **Crucial for mapping** and spatial clustering analysis.                                            |

---

#### 1. **Visualize Businesses on a Map**

* Use tools like **Folium**, **Kepler.gl**, or **GeoPandas + Matplotlib** to:

  * Visualize the distribution of businesses across Ozurgeti.
  * Color-code by `Name of activity` or `Business size`.
  * Add interactivity (e.g., tooltips with `Name of entity` or legal form).

#### 2. **Spatial Clustering**

* Use clustering algorithms like:

  * **DBSCAN** to find business hotspots (e.g., market centers or industrial zones).
  * **K-Means** (with lat/long or UTM) to segment the city into business zones.
* Identify areas with **no business presence** — potential zones for development.

#### 3. **Sectoral Distribution by Area**

* Aggregate businesses by `Name of activity` + location (maybe via spatial bins or grid).
* Answer questions like:

  * Where are most service-based businesses located?
  * Are retail businesses clustered around city center or spread?

#### 4. **Business Density Heatmaps**

* Create heatmaps showing:

  * **Total business density**.
  * **Density of certain types** (e.g., restaurants, pharmacies, construction).
* Use this to identify:

  * Underserved neighborhoods.
  * Areas of over-saturation or imbalance.

#### 5. **Compare Legal Forms Across the City**

* Analyze how organizational-legal forms (LLC vs sole proprietors) vary spatially.
* May indicate formality/informality of the economy by zone.

#### 6. **Overlay with External Layers**

* Integrate with:

  * Population density,
  * Transport infrastructure (roads, bus stops),
  * Public services (schools, hospitals),
  * Urban zoning or land use maps (if available).
* This helps assess whether current business distribution aligns with urban accessibility.

#### 7. **Development Potential Score (Custom Index)**

* Create a composite metric for each area/grid:

  * Factors: current business density, business diversity, distance to roads, proximity to population centers, etc.
* Rank or color-code the city’s zones for **investment or development potential**.

---

### 🛠️ Tools 

* **Python Libraries:**

  * `pandas`, `geopandas`, `matplotlib`, `seaborn`, `folium`, `scikit-learn`
* **Interactive Visualization:**

  * `kepler.gl`, `Plotly`, `Leaflet.js`, or GIS tools like **QGIS**
* **Web Map (if building an app):**

  * `Mapbox`, `Leaflet`, or `Deck.gl` for powerful front-end mapping.

---




https://edm.gv.at/edm_portal/redaList.do?d-49520-s=1&ubastyle_lang=en&d-49520-p=7&seqCode=g5zjkt69fs9j33&search=&d-49520-o=2

https://ec.europa.eu/eurostat/documents/3859598/5902521/KS-RA-07-015-EN.PDF

In [85]:
import pandas as pd
import folium

In [86]:
data = pd.read_excel("Ozurgeti_dataset_coordinates_filtered.xlsx")
NACE_REV2 = pd.read_excel("NACE Rev2.xlsx")

In [87]:
data

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,საქმიანობის კოდი NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა,ნომერი,ქუჩა,ქუჩა_OPS,similarity_score,matched_street,ქუჩა_საბოლოო,St_Full_Name,კოორდინატები,Long,Lat
0,100128238,3.300101e+10,NaN,იმ,ლევან ყარგიშვილი,გრიბოედოვის ქ31 ბ32,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",47.11.0,აქტიური,მცირე,3132,გრიბოედოვის,ალექსანდრე გრიბოედოვის,100.0,ალექსანდრე გრიბოედოვის,ალექსანდრე გრიბოედოვის,"3132 ალექსანდრე გრიბოედოვის ქუჩა, Ozurgeti, Ge...","('3132 ალექსანდრე გრიბოედოვის ქუჩა, Ozurgeti, ...",41.920735,41.999584
1,102370195,3.300103e+10,NaN,იმ,სოფიო ღლონტი,ჯორჯიაშვილის1,NaN,NaN,NaN,NaN,1,ჯორჯიაშვილის,ჯორჯიაშვილის,100.0,ჯორჯიაშვილის,ჯორჯიაშვილის,"1 ჯორჯიაშვილის ქუჩა, Ozurgeti, Georgia","('1 ჯორჯიაშვილის ქუჩა, Ozurgeti, Georgia', 41....",41.924247,41.983653
2,102398219,3.300103e+10,ქ. ოზურგეთი,იმ,ელისო ქადარია,ბათუმის 3,სხვა საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში,47.19.0,აქტიური,მცირე,3,ბათუმის,ბათუმის,100.0,ბათუმის,ბათუმის,"3 ბათუმის ქუჩა, Ozurgeti, Georgia","('3 ბათუმის ქუჩა, Ozurgeti, Georgia', 41.93070...",41.930701,41.993382
3,102399940,3.300103e+10,NaN,იმ,თამარი გიორგაძე,ერისთავის N13 ბ,საცალო ვაჭრობა სხვა ნახმარი საქონლით,47.79.4,აქტიური,მცირე,13,ერისთავის,დიმიტრი ერისთავის,100.0,დიმიტრი ერისთავის,დიმიტრი ერისთავის,"13 დიმიტრი ერისთავის ქუჩა, Ozurgeti, Georgia","('13 დიმიტრი ერისთავის ქუჩა, Ozurgeti, Georgia...",41.923523,42.000153
4,104430894,3.300101e+10,NaN,იმ,დავით ყანდიაშვილი-აუდიენცია,ნ. ნიკოლაძის ქ. N24,NaN,NaN,NaN,NaN,24,ნიკოლაძის,ნიკოლაძის,100.0,ნიკოლაძის,ნიკოლაძის,"24 ნიკოლაძის ქუჩა, Ozurgeti, Georgia","('24 ნიკოლაძის ქუჩა, Ozurgeti, Georgia', 41.92...",41.920805,41.993133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1735,61010006946,6.101001e+10,ქ. ოზურგეთი,იმ,ირა ირემაძე,გურიის ქუჩა #35,რესტორნები და საკვებით მობილური მომსახურების ს...,56.10.0,აქტიური,მცირე,35,გურიის,გურიის,100.0,გურიის,გურიის,"35 გურიის ქუჩა, Ozurgeti, Georgia","('35 გურიის ქუჩა, Ozurgeti, Georgia', 41.92190...",41.921904,42.001573
1736,61010016559,6.101002e+10,NaN,იმ,ნათელა თოიძე,აღმაშენებლის ქ.N21,საცალო ვაჭრობა სხვა საქონლით ფარდულებში და ბაზ...,47.89.0,აქტიური,მცირე,21,აღმაშენებლის,აღმაშენებლის,100.0,აღმაშენებლის,აღმაშენებლის,"21 აღმაშენებლის ქუჩა, Ozurgeti, Georgia","('21 აღმაშენებლის ქუჩა, Ozurgeti, Georgia', 41...",41.920763,42.008140
1737,62006018863,6.200602e+10,ქ. ოზურგეთი,იმ,ხათუნა ელიავა,აღმაშენებლის ქუჩა #10,საცალო ვაჭრობა თამბაქოს ნაწარმით სპეციალიზებულ...,47.26.0,NaN,NaN,10,აღმაშენებლის,აღმაშენებლის,100.0,აღმაშენებლის,აღმაშენებლის,"10 აღმაშენებლის ქუჩა, Ozurgeti, Georgia","('10 აღმაშენებლის ქუჩა, Ozurgeti, Georgia', 41...",41.921619,42.007026
1738,62006022557,6.200602e+10,NaN,იმ,გიზო ჯალაღონია,გურიის ქუჩა N35,საცხოვრებელი და არასაცხოვრებელი შენობების მშენ...,41.20.0,NaN,NaN,35,გურიის,გურიის,100.0,გურიის,გურიის,"35 გურიის ქუჩა, Ozurgeti, Georgia","('35 გურიის ქუჩა, Ozurgeti, Georgia', 41.92190...",41.921904,42.001573


In [88]:
data[['Code1','Code2','Code3']]=data['საქმიანობის კოდი NACE Rev.2'].str.split(pat='.', expand=True)

In [89]:
data.drop(columns=['Code2', 'Code3'], inplace=True)

In [90]:
NACE_REV2['Rev 2.1 code']=NACE_REV2['Rev 2.1 code'].astype(str)

In [91]:
data=data.merge(NACE_REV2, left_on='Code1', right_on='Rev 2.1 code',how='left')

In [92]:
Columns=['პირადი ნომერი', 
                   'დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)',
                   'ფაქტობრივი მისამართი',
                   'აქტიური ეკონომიკური სუბიექტები',
                   'Rev 2.1 code',
                    'Rev 2 code',
                    'საქმიანობის დასახელება NACE Rev.2',
                    'საქმიანობის კოდი NACE Rev.2',
                    'Code1',
                    'ნომერი',
                    'ქუჩა',
                    'ქუჩა_OPS',
                    'similarity_score',
                    'matched_street',
                    'ქუჩა_საბოლოო',
                    'კოორდინატები'
                    ]
Droppable = [col for col in Columns if col in data.columns]
data.drop(columns=Droppable, inplace=True)

In [93]:
data.head(4)

,საიდენტიფიკაციო ნომერი,ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ბიზნესის ზომა,St_Full_Name,Long,Lat,Name
0,100128238,იმ,ლევან ყარგიშვილი,მცირე,"3132 ალექსანდრე გრიბოედოვის ქუჩა, Ozurgeti, Ge...",41.920735,41.999584,Retail trade
1,102370195,იმ,სოფიო ღლონტი,NaN,"1 ჯორჯიაშვილის ქუჩა, Ozurgeti, Georgia",41.924247,41.983653,NaN
2,102398219,იმ,ელისო ქადარია,მცირე,"3 ბათუმის ქუჩა, Ozurgeti, Georgia",41.930701,41.993382,Retail trade
3,102399940,იმ,თამარი გიორგაძე,მცირე,"13 დიმიტრი ერისთავის ქუჩა, Ozurgeti, Georgia",41.923523,42.000153,Retail trade


In [94]:
df=pd.DataFrame(data[~data['Name'].isna()].value_counts('Name')).reset_index()

In [95]:
Top_10=df.sort_values(by='count', ascending=False).head(10)

In [96]:
Top_10

,Name,count
0,Retail trade,660
1,Wholesale trade,105
2,Manufacture of food products,58
3,Wholesale and retail trade and repair of motor...,54
4,Real estate activities,48
5,Food and beverage service activities,45
6,Land transport and transport via pipelines,42
7,Personal service activities,29
8,Construction of residential and non-residentia...,28
9,Specialised construction activities,26


In [97]:
df_top_10 = data[data['Name'].isin(Top_10['Name'])]

In [129]:
import folium

# 1. Define your colors
colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'skyblue', 'pink', 'gray', 'brown']

# 2. Map each category to a color
unique_names = df_top_10['Name'].unique()
name_to_color = {name: colors[i] for i, name in enumerate(unique_names)}

# 3. Create the map
ozurgeti_map = folium.Map(
    location=[41.925, 42.003],
    tiles='CartoDB positron',
    zoom_start=14.5,
)

# 4. Add markers
for idx, row in df_top_10.iterrows():
    folium.CircleMarker(
        location=[row['Long'], row['Lat']],  # Note: folium uses Lat, Long order
        radius=5,
        color=name_to_color.get(row['Name'], 'white'),  # fallback color
        fill=True,
        fill_color=name_to_color.get(row['Name'], 'white'),
        fill_opacity=0.6,
        popup=row['Name'],
        tooltip=row['Name']
    ).add_to(ozurgeti_map)

# 5. Display the map
display(ozurgeti_map)



In [120]:
name_to_color

{'Retail trade ': 'red',
 'Food and beverage service activities ': 'blue',
 'Construction of residential and non-residential buildings ': 'green',
 'Manufacture of food products ': 'purple',
 'Real estate activities ': 'orange',
 'Wholesale trade ': 'yellow',
 'Wholesale and retail trade and repair of motor vehicles and motorcycles': 'skyblue',
 'Personal service activities ': 'pink',
 'Land transport and transport via pipelines ': 'gray',
 'Specialised construction activities ': 'brown'}